In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

In [2]:
df = pd.read_stata('comp_dr.dta')
df

,gvkey,datadate,indfmt,consol,popsrc,datafmt,tic,conm,curcd,fyr,dlc,drc,drlt,lt,costat,LINKTYPE,LPERMNO,MthCalDt
0,001000,1971-12-31,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,12.0,2.857,NaN,NaN,20.948,I,LU,25881.0,1971-12-31
1,001000,1972-12-31,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,12.0,0.000,NaN,NaN,12.886,I,LU,25881.0,1972-12-29
2,001000,1973-12-31,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,12.0,0.000,NaN,NaN,13.204,I,LU,25881.0,1973-12-31
3,001000,1974-12-31,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,12.0,0.500,NaN,NaN,15.381,I,LU,25881.0,1974-12-31
4,001000,1975-12-31,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,12.0,1.251,NaN,NaN,11.596,I,LU,25881.0,1975-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305588,349972,2022-12-31,INDL,C,D,STD,INDP,INDAPTUS THERAPEUTICS INC,USD,12.0,0.080,0.00,0.0,3.433,A,LC,15642.0,2022-12-30
305589,349972,2023-12-31,INDL,C,D,STD,INDP,INDAPTUS THERAPEUTICS INC,USD,12.0,0.102,0.00,0.0,2.847,A,LC,15642.0,2023-12-29
305590,351038,2022-12-31,INDL,C,D,STD,QNRX,QUOIN PHARMACEUTICALS LTD,USD,12.0,0.000,0.00,0.0,7.051,A,LC,16161.0,2022-12-30
305591,351038,2023-12-31,INDL,C,D,STD,QNRX,QUOIN PHARMACEUTICALS LTD,USD,12.0,0.000,0.00,0.0,6.505,A,LC,16161.0,2023-12-29


In [3]:
#current deferred revenue - drc
#total current liabilities - dlc
#long-term deferred revenue - drlt
#total long-term liabilities - lt
df = df[['gvkey', 'datadate', 'tic', 'drc', 'dlc', 'drlt', 'lt']]
df['ratio'] = df['drc'] / df['lt']
df['datadate'] = pd.to_datetime(df['datadate'])

df['datadate'] = df['datadate'].dt.strftime('%Y-%m-%d')
df

C:\Users\the1s\AppData\Local\Temp\ipykernel_29244\1504043774.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ratio'] = df['drc'] / df['lt']
C:\Users\the1s\AppData\Local\Temp\ipykernel_29244\1504043774.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datadate'] = pd.to_datetime(df['datadate'])
C:\Users\the1s\AppData\Local\Temp\ipykernel_29244\1504043774.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,gvkey,datadate,tic,drc,dlc,drlt,lt,ratio
0,001000,1971-12-31,AE.2,NaN,2.857,NaN,20.948,NaN
1,001000,1972-12-31,AE.2,NaN,0.000,NaN,12.886,NaN
2,001000,1973-12-31,AE.2,NaN,0.000,NaN,13.204,NaN
3,001000,1974-12-31,AE.2,NaN,0.500,NaN,15.381,NaN
4,001000,1975-12-31,AE.2,NaN,1.251,NaN,11.596,NaN
...,...,...,...,...,...,...,...,...
305588,349972,2022-12-31,INDP,0.00,0.080,0.0,3.433,0.000000
305589,349972,2023-12-31,INDP,0.00,0.102,0.0,2.847,0.000000
305590,351038,2022-12-31,QNRX,0.00,0.000,0.0,7.051,0.000000
305591,351038,2023-12-31,QNRX,0.00,0.000,0.0,6.505,0.000000


In [4]:

base = alt.Chart(df).encode(
    x = alt.X('datadate', title='Date')
)

layered_chart = alt.layer(
    base.mark_line(color='blue').encode(y='dlc', title='Value of dlc'),
    base.mark_line(color='red').encode(y='drc', title='Value of drc'),
    base.mark_line(color='green').encode(y='drlt', title='Value of drlt'),
    base.mark_line(color='orange').encode(y='lt', title='Value of lt')
).resolve_scale(y='independent')


layered_chart

C:\Users\the1s\miniconda3\Lib\site-packages\altair\utils\core.py:714: UserWarning: Unrecognized encoding channel 'title'
  warnings.warn(


MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

See https://altair-viz.github.io/user_guide/large_datasets.html for information on how to plot large datasets, including how to install third-party data management tools and, in the right circumstance, disable the restriction

alt.LayerChart(...)